In [1]:
import cv2
import numpy as np
import os

In [2]:
def fill_color_demo(image):
    copyIma = image.copy()
    h, w = image.shape[:2]
    print(h, w)
    mask = np.zeros([h+2, w+2], np.uint8)
    
    cv2.floodFill(copyIma, mask, (0, 0), (0, 0, 0), (100, 100, 100), (50, 50, 50), cv2.FLOODFILL_FIXED_RANGE)
    cv2.floodFill(copyIma, mask, (w-1, 0), (0, 0, 0), (100, 100, 100), (50, 50, 50), cv2.FLOODFILL_FIXED_RANGE)  
    cv2.floodFill(copyIma, mask, (0, h-1), (0, 0, 0), (100, 100, 100), (50, 50, 50), cv2.FLOODFILL_FIXED_RANGE)  
    cv2.floodFill(copyIma, mask, (w-1, h-1), (0, 0, 0), (100, 100, 100), (50, 50, 50), cv2.FLOODFILL_FIXED_RANGE)  

    return copyIma

In [3]:
source_path = "../../jpg_files/Experiment_Graphs/Original"
result_path = "../../jpg_files/Experiment_Graphs/CCL_Results"

source_sparse = "Sparse_KKW_n=2000_d=6.jpg"
source_uniform = "Uniform_KKW_n=2000_d=6.jpg"
target_sparse = "Sparse_KKW_n=2000_d=6_Result_CCL.jpg"
target_uniform = "Uniform_KKW_n=2000_d=6_Result_CCL.jpg"
target_ff_uniform = "Uniform_KKW_n=2000_d=6_FF_CCL.jpg"
target_ff_sparse = "Sparse_KKW_n=2000_d=6_FF_CCL.jpg"
# 读入图片
img = cv2.imread(os.path.join(source_path, source_sparse))
# img = cv2.imread(os.path.join(source_path, source_uniform))

img_ff = fill_color_demo(img)
# cv2.imwrite(os.path.join(result_path, target_ff_sparse), img_ff)

8000 8000


In [4]:
# 中值滤波，去噪
img_ff = cv2.medianBlur(img_ff, 3)
gray = cv2.cvtColor(img_ff, cv2.COLOR_BGR2GRAY)
# cv2.imwrite(os.path.join(result_dir, grey_file), gray)

# 阈值分割得到二值化图片
ret, binary = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
# cv2.imwrite(os.path.join(result_dir, binary_file), binary)
# 膨胀操作
# kernel2 = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
# bin_clo = cv2.dilate(binary, kernel2, iterations=2)

# 连通域分析
num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(binary, connectivity=4)
# num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(bin_clo, connectivity=8)

# 查看各个返回值
# 连通域数量
print('num_labels = ',num_labels)
# 连通域的信息：对应各个轮廓的x、y、width、height和面积
print('stats = ',stats)
# 连通域的中心点
print('centroids = ',centroids)
# 每一个像素的标签1、2、3.。。，同一个连通域的标签是一致的
print('labels = ',labels)

# 不同的连通域赋予不同的颜色
# output = np.zeros((img.shape[0], img.shape[1], 3), np.uint8)
# output[:, :] = [255, 255, 255] 
output = img
for i in range(1, num_labels):
    # Area of the component
    area = stats[i, cv2.CC_STAT_AREA] 
    if area > 10000:
#         print(str(i) + " area is " + str(area))
        mask = labels == i
#         output[:, :, 0][mask] = np.random.randint(0, 255)
#         output[:, :, 1][mask] = np.random.randint(0, 255)
#         output[:, :, 2][mask] = np.random.randint(0, 255)
        output[:, :, 0][mask] = 255
        output[:, :, 1][mask] = 0
        output[:, :, 2][mask] = 0

#         componentMask = (label_ids == i).astype("uint8") * 255
#         output = cv2.bitwise_or(output, componentMask)
        
# cv2.imwrite(os.path.join(result_path, target_uniform), output)
cv2.imwrite(os.path.join(result_path, target_sparse), output)

num_labels =  3090
stats =  [[       0        0     8000     8000 40015068]
 [    4953       14       99       39     1786]
 [    4905       24       32       36      599]
 ...
 [    3943     7593        5        6       20]
 [    4075     7602       46       36      784]
 [    3957     7605       27        9      150]]
centroids =  [[3902.80263567 4212.58700825]
 [4993.79339306   34.27547592]
 [4923.11853088   45.8196995 ]
 ...
 [3944.45       7595.4       ]
 [4094.32397959 7618.84821429]
 [3969.34       7610.24      ]]
labels =  [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


True